In [ ]:
#貌似Miniconda在安装时会自动下载安装Python3.12.8版本open-webui需要python>=3.11,而在kaggle里面安装python并配置环境变量又不怎么容易所以就直接使用Miniconda
#安装脚本来安装python3.12.8并配置环境变量
#kaggle截止2025/2/5使用的python版本依然是3.10.9,安装python3.12.8
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local

In [ ]:
!pip install open-webui
!curl -fsSL https://ollama.com/install.sh | sh
!npm install -g localtunnel

In [ ]:
import subprocess
import threading
import time
import socket
from colorama import Fore, Style
import urllib.request

model_name = "deepseek-r1:14b"

def read_stream(pipe, handler):
    """Read stream from pipe and apply the handler function."""
    try:
        for line in iter(pipe.readline, ''):
            if line:
                handler(line.strip())
    except ValueError:
        # 忽略管道关闭时可能抛出的异常
        pass
    finally:
        pipe.close()

def run_ollama_task():
    print("Ollama task started")
    # 使用 subprocess.Popen 启动 ollama serve 命令
    process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    time.sleep(5)
    pull_model = subprocess.Popen(["ollama", "pull", model_name], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # 创建线程来读取输出和错误信息
    t_out = threading.Thread(target=read_stream, args=(pull_model.stdout, print))
    t_err = threading.Thread(target=read_stream, args=(pull_model.stderr, lambda x: print(Fore.RED + x + Style.RESET_ALL)))
    t_out.daemon = True
    t_err.daemon = True
    t_out.start()
    t_err.start()

    pull_model.wait()  # 等待模型拉取完成
    if pull_model.returncode == 0:
        print(Fore.GREEN + "Model download completed" + Style.RESET_ALL)
    else:
        print(Fore.RED + "Model download failed" + Style.RESET_ALL)

    list_models = subprocess.Popen(["ollama", "list"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    read_stream(list_models.stdout, print)  # 实时捕获并处理列表命令的输出
    list_models.wait()
    print(Fore.GREEN + "Ollama task is running in the background" + Style.RESET_ALL)

def run_open_webui():
    print("open-webui is starting...")
    process2 = subprocess.Popen(['open-webui', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    read_stream(process2.stdout, print)
    process2.wait()
    print(Fore.GREEN + "open-webui task is running in the background" + Style.RESET_ALL)

def iframe_thread(port):
    # 等待端口打开
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            sock.close()
            break
        sock.close()

    # 获取IP地址
    try:
        public_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip()
        ip_message = f"{Fore.GREEN}\nIP: {Fore.RED}{public_ip}{Style.RESET_ALL}\n"
    except Exception as e:
        ip_message = f"{Fore.RED}\nError fetching public IP: {str(e)}{Style.RESET_ALL}\n"
    
    print(ip_message)

    # 启动隧道
    p = subprocess.Popen(["lt", "--port", str(port)], stdout=subprocess.PIPE, text=True)
    for line in p.stdout:
        print(line.strip())

# 在后台线程中运行 Ollama 任务
ollama_thread = threading.Thread(target=run_ollama_task)
ollama_thread.start()
# 等待 Ollama 线程完成
ollama_thread.join()
threading.Thread(target=run_open_webui).start()

threading.Thread(target=iframe_thread, daemon=True, args=(8080,)).start()